## Tratamento dos dados

### Importando bibliotecas

In [3]:
import numpy as np
from sklearn.metrics import adjusted_rand_score
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import pairwise_distances

### Carregando o dataset

In [4]:
def gerar_configuracao(mu_list, sigma_list, tamanhos, config_id):
    dfs = []
    for i, (mu, sigma2, n) in enumerate(zip(mu_list, sigma_list, tamanhos)):
        Sigma = np.diag(sigma2)
        data = np.random.multivariate_normal(mu, Sigma, n)
        df = pd.DataFrame(data, columns=["x1", "x2"])
        df["class"] = i + 1
        dfs.append(df)
    df_config = pd.concat(dfs, ignore_index=True)
    df_config["config"] = config_id
    return df_config

np.random.seed(42)  # reprodutibilidade

# -------------------------------
# Configuração 1
mu_1 = [[5, 0], [15, 5], [18, 14]]
sigma2_1 = [[81, 9], [9, 100], [25, 36]]
n1 = [200, 100, 50]
df1 = gerar_configuracao(mu_1, sigma2_1, n1, config_id=1)
l1 = "Classes elípticas de tamanhos diferentes"

# -------------------------------
# Configuração 2
mu_2 = [[0, 0], [30, 0], [12, 25]]
sigma2_2 = [[100, 100], [49, 49], [16, 16]]
n2 = [200, 100, 50]
df2 = gerar_configuracao(mu_2, sigma2_2, n2, config_id=2)
l2 = "Classes esféricas de tamanhos diferentes"

# -------------------------------
# Configuração 3
mu_3 = [[0, 0], [15, 5], [15, -5]]
sigma2_3 = [[100, 4], [100, 4], [100, 4]]
n3 = [100, 100, 100]
df3 = gerar_configuracao(mu_3, sigma2_3, n3, config_id=3)
l3 = "Classes elípticas de tamanhos iguais"

# -------------------------------
# Configuração 4
mu_4 = [[0, 0], [15, 0], [-15, 0]]
sigma2_4 = [[16, 16], [16, 16], [16, 16]]
n4 = [100, 100, 100]
df4 = gerar_configuracao(mu_4, sigma2_4, n4, config_id=4)
l4 = "Classes elípticas de tamanhos iguais"

# -------------------------------
# Configuração 5
mu_5 = [[5, 0], [15, 5], [10, -7], [3, 15]]
sigma2_5 = [[81, 9], [9, 100], [49, 16], [25, 25]]
n5 = [50, 50, 50, 50]
df5 = gerar_configuracao(mu_5, sigma2_5, n5, config_id=5)
l5 = "3 classes elípticas e 1 esférica"

# -------------------------------
# Configuração 6
mu_6 = [[5, 0], [15, 5], [12, -12], [7, 17]]
sigma2_6 = [[81, 9], [9, 100], [16, 16], [25, 25]]
n6 = [50, 50, 50, 50]
df6 = gerar_configuracao(mu_6, sigma2_6, n6, config_id=6)
l6 = "2 classes elípticas e 2 esféricas"

# -------------------------------
# Configuração 7
mu_7 = [[0, 0], [18, 0], [-18, 0], [0, -12]]
sigma2_7 = [[12, 12], [20, 20], [16, 16], [81, 20]]
n7 = [50, 50, 50, 50]
df7 = gerar_configuracao(mu_7, sigma2_7, n7, config_id=7)
l7 = "1 classe elíptica e 3 esféricas"

### Transformando classes em números

In [ ]:
def crisp_to_fuzzy(y, n_clusters): # transforma o dataset em fuzzy
    fuzzy_labels = np.zeros((len(y), n_clusters)) # cria uma array do dataset preenchida só com zeros
    for i, label in enumerate(y):
        fuzzy_labels[i, label] = 1 # com base na classe, o zero é substituído por um
    return fuzzy_labels

def init_membership_matrix(n, k):
    membership_matrix = np.random.rand(n, k) # gera uma matriz inicial aleatória com valores entre 0 e 1
    membership_matrix = membership_matrix / membership_matrix.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return membership_matrix

def init_medoids(X, c):
    total_distances = np.sum(pairwise_distances(X), axis=1) # distância somada de cada ponto para os outros
    
    first_medoid_idx = np.argmin(total_distances) # ponto com menor distância total
    medoids_indices = [first_medoid_idx]  # lista para os índices dos medoides
    medoids = [X[first_medoid_idx]]  # lista para os medoides

    for _ in range(1, c): # (1, c) pois já temos um medoide
        max_min_dist = -np.inf # armazena a maior entre as menores distâncias
        next_medoid_idx = -1 # armazena o índice do candidato a medoide

        for i in range(len(X)):
            if i in medoids_indices: # ignora os pontos que já são medoides
                continue 

            # primeiro ele calcula a distância do ponto para cada um dos medoides
            # depois ele seleciona a menor dessas distâncias
            min_dist = np.min([pairwise_distances(X[i].reshape(1, -1), np.array([medoid])).flatten()[0] for medoid in medoids])

            if min_dist > max_min_dist: # a maior distância entre as menores
                max_min_dist = min_dist
                next_medoid_idx = i

        # adiciona o ponto escolhido como medoide
        medoids_indices.append(next_medoid_idx)
        medoids.append(X[next_medoid_idx])

    return np.array(medoids)

def update_membership_matrix(data, medoids, m):
    distance_matrix = pairwise_distances(data, medoids, metric='euclidean') ** 2
    distance_matrix = np.fmax(distance_matrix, np.finfo(np.float64).eps)  # evita que matriz_distancias seja 0, np.finfo... é o menor número maior que zero aqui
    
    inverse_distance_matrix = 1 / distance_matrix
    power = 1 / (m - 1)
    updated_membership_matrix = (inverse_distance_matrix ** power) / np.sum(inverse_distance_matrix ** power, axis=1, keepdims=True) # fórmula para atualizar os graus de pertinência
    
    return updated_membership_matrix

def update_medoids(X, medoids, membership_matrix, m=2):
    n, c = X.shape[0], len(medoids)
    updated_medoids = np.copy(medoids)  # cópia dos medoides 
    chosen_medoids_indices = []  # armazena os índices dos novos medoides
    distances = pairwise_distances(X, X)  # matriz de distância entre os pontos

    for i in range(c): 
        initial_medoid_idx = np.where((X == medoids[i]).all(axis=1))[0][0]  # índice do medoide atual
        min_weighted_distance = np.sum([(membership_matrix[k, i] ** m) * distances[k, initial_medoid_idx] for k in range(n)]) # distância ponderada do medoide atual
        best_medoid = medoids[i] # melhor candidato a medoide
        best_medoid_idx = initial_medoid_idx # índice do melhor candidato a medoide

        for j in range(n):
            if j in chosen_medoids_indices:  # ignorar pontos que já são medoides
                continue
            
            weighted_distance = np.sum([(membership_matrix[k, i] ** m) * distances[k, j] for k in range(n)]) # distância ponderada do candidato a medoide

            if weighted_distance < min_weighted_distance: # caso a distância ponderada do candidato seja menor que a do medoide atual
                min_weighted_distance = weighted_distance # atualiza a menor distância ponderada
                best_medoid = X[j] # atualiza o melhor candidato a medoide
                best_medoid_idx = j # atualiza o índice do melhor candidato a medoide
                updated_medoids[i] = best_medoid # já coloca ele como medoide

        updated_medoids[i] = best_medoid # atualiza de fato os medoides
        chosen_medoids_indices.append(best_medoid_idx) # para garantir que ele não será escolhido como medoide novamente

    return updated_medoids

def fcmdd(data, k, m=2, max_iter=1000000):
    n = data.shape[0]
    membership_matrix = init_membership_matrix(n, k)
    medoids = init_medoids(data, k)
    for _ in range(max_iter):
        membership_matrix = update_membership_matrix(data, medoids, m)
        new_medoids = update_medoids(data, medoids, membership_matrix, m)
        if np.array_equal(medoids, new_medoids): # se os medoides não mudaram, para
            break
        medoids = new_medoids
    return medoids, membership_matrix

def pertinence_distance(delta_k, delta_k_linha, c):
    # calcula a distância entre δ_k e δ_k' (matrizes de pertinência)
    return (1/c) * np.sum((delta_k - delta_k_linha) ** 2)

def fuzzy_rand_index(particao1, particao2, c):
    n = particao1.shape[0]
    total_sum = 0

    for k in range(n):
        for k_linha in range(k+1, n):
            if k != k_linha:
                # calcula a métrica para P
                delta_k = particao1[k]
                delta_k_prime = particao1[k_linha]
                EP = 1 - pertinence_distance(delta_k, delta_k_prime, c)

                # calcula a métrica para Q
                delta_k_Q = particao2[k]
                delta_k_prime_Q = particao2[k_linha]
                EQ = 1 - pertinence_distance(delta_k_Q, delta_k_prime_Q, c)

                total_sum += np.abs(EP - EQ) # soma a diferença absoluta entre EP e EQ

    denominador = n * (n - 1) / 2
    if denominador == 0:
        raise ValueError

    return 1- (total_sum / denominador)

def monte_carlo_fuzzy_simulation(X, true_labels, k, m=2, num_trials=100):
    results = []
    for trial in range(num_trials):
        medoids, membership_matrix = fcmdd(X, k, m)
        predicted_labels = np.argmax(membership_matrix, axis=1)
        rand_idx = fuzzy_rand_index(true_labels, predicted_labels, k)
        results.append(rand_idx)
    
    mean_ari = np.mean(results)
    std_ari = np.std(results)
    return mean_ari, std_ari

i = 1
for df in [df1, df2, df3, df4, df5, df6, df7]:
    if i == 5 or i == 6 or i == 7:
        num_clusters = 4
    else:
        num_clusters = 3
    df.drop("config", axis=1, inplace=True)
    if i == 5 or i == 6 or i == 7:
        df["class"].replace({1: 0, 2: 1, 3: 2, 4: 3}, inplace=True)
    else: 
        df["class"].replace({1: 0, 2: 1, 3: 2}, inplace=True)
    labels = df["class"].values
    df.drop("class", axis=1, inplace=True)
    data = df.to_numpy()
    labels = crisp_to_fuzzy(labels, num_clusters)
    k = num_clusters
    num_trials = 100
    m = 2
    mean_rand_index, std_rand_index = monte_carlo_fuzzy_simulation(data, labels, k, m, num_trials)

    print(f"Monte Carlo FCMdd Clustering Results for Config {i}")
    print(f"Mean Rand Index: {mean_rand_index:.4f}")
    print(f"Standard Deviation of Rand Index: {std_rand_index:.4f}")
    print("\n")
    i += 1

## Resultados:

Monte Carlo FCMdd Clustering Results for Config 1  
Mean Rand Index: 0.4858  
Standard Deviation of Rand Index: 0.0000  


Monte Carlo FCMdd Clustering Results for Config 2  
Mean Rand Index: 0.5425  
Standard Deviation of Rand Index: 0.0000  


Monte Carlo FCMdd Clustering Results for Config 3  
Mean Rand Index: 0.4962  
Standard Deviation of Rand Index: 0.0000  


Monte Carlo FCMdd Clustering Results for Config 4  
Mean Rand Index: 0.6573  
Standard Deviation of Rand Index: 0.0000  


Monte Carlo FCMdd Clustering Results for Config 5  
Mean Rand Index: 0.5711  
Standard Deviation of Rand Index: 0.0000  


Monte Carlo FCMdd Clustering Results for Config 6  
Mean Rand Index: 0.4165  
Standard Deviation of Rand Index: 0.0000  


Monte Carlo FCMdd Clustering Results for Config 7  
Mean Rand Index: 0.5462  
Standard Deviation of Rand Index: 0.0000  

**O código foi rodado no cluster Apuana CIn-UFPE**